# <h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [ ]:
import pandas as pd
data = pd.read_csv('Churn.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


**Признаки**<br>
RowNumber — индекс строки в данных<br>
CustomerId — уникальный идентификатор клиента<br>
Surname — фамилия<br>
CreditScore — кредитный рейтинг<br>
Geography — страна проживания<br>
Gender — пол<br>
Age — возраст<br>
Tenure — сколько лет человек является клиентом банка<br>
Balance — баланс на счёте<br>
NumOfProducts — количество продуктов банка, используемых клиентом<br>
HasCrCard — наличие кредитной карты<br>
IsActiveMember — активность клиента<br>
EstimatedSalary — предполагаемая зарплата<br>
**Целевой признак**<br>
Exited — факт ухода клиента<br>

In [ ]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

Удалила не нужныые столбцы, эта информация не имеет значения в жанном иссследовании

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           9091 non-null   float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


In [ ]:
data[data['Tenure'].isna()]

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...
9944,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


закономерности или взаимосвязи с другими данными не наблюдаю, поэтому заполню пропуски медианныи значением<br>
они могли появиться или из за технической ошибки или можкет это означает, что клиент только пришел

In [ ]:
data['Tenure'] = data['Tenure'].fillna(data['Tenure'].median())

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


In [ ]:
categorical_features = ['Geography', 'Gender']

numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance',
                   'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

Преобразую категориальные признаки в численные, с помощью One-Hot Encoding, так как Gender и Geography  не имеют значимой последовательности

In [ ]:
data = pd.get_dummies(data, drop_first= True).astype(int)
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0,1,1,1,101348,1,0,0,0
1,608,41,1,83807,1,0,1,112542,0,0,1,0
2,502,42,8,159660,3,1,0,113931,1,0,0,0
3,699,39,1,0,2,0,0,93826,0,0,0,0
4,850,43,2,125510,1,1,1,79084,0,0,1,0


Стандартизирую признаки

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,-0.326221,0.293517,-1.086246,-1.225847,-0.911583,0.646092,0.970243,0.021880,1,0,0,0
1,-0.440036,0.198164,-1.448581,0.117342,-0.911583,-1.547768,0.970243,0.216532,0,0,1,0
2,-1.536794,0.293517,1.087768,1.333050,2.527057,0.646092,-1.030670,0.240686,1,0,0,0
3,0.501521,0.007457,-1.448581,-1.225847,0.807737,-1.547768,-1.030670,-0.108920,0,0,0,0
4,2.063884,0.388871,-1.086246,0.785723,-0.911583,0.646092,0.970243,-0.365269,0,0,1,0


теперь данные готовы к дальнейшему исследованиб

## Исследование задачи

выделим целевую пременную и аргументы

In [ ]:
features = data.drop(['Exited'], axis=1)
features.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,-0.326221,0.293517,-1.086246,-1.225847,-0.911583,0.646092,0.970243,0.021880,0,0,0
1,-0.440036,0.198164,-1.448581,0.117342,-0.911583,-1.547768,0.970243,0.216532,0,1,0
2,-1.536794,0.293517,1.087768,1.333050,2.527057,0.646092,-1.030670,0.240686,0,0,0
3,0.501521,0.007457,-1.448581,-1.225847,0.807737,-1.547768,-1.030670,-0.108920,0,0,0
4,2.063884,0.388871,-1.086246,0.785723,-0.911583,0.646092,0.970243,-0.365269,0,1,0


In [ ]:
target = data['Exited']
target.head()

,Exited
0,1
1,0
2,1
3,0
4,0


для начала разделим данные на три выборки: train, valid и test

In [ ]:
from sklearn.model_selection import train_test_split
features_full_train, features_test, target_full_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_full_train, target_full_train, test_size=0.25, random_state=12345)

In [ ]:
class_dist = target_train.value_counts(normalize=True)
class_dist

,proportion
Exited,
0,0.800533
1,0.199467


хорошо видно, что присутствует дисбаланс классов

обучим разные модели без учета дисбаланса класса

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score

model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_train, target_train)
predict_f = model.predict(features_valid)
f_1 = f1_score(target_valid, predict_f)
roc_auc = roc_auc_score(target_valid, predict_f)
print(f'f1: {f_1}')

f1: 0.31640625


In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state = 12345)
model.fit(features_train, target_train)
predict_f = model.predict(features_valid)
f_1 = f1_score(target_valid, predict_f)
roc_auc = roc_auc_score(target_valid, predict_f)
print(f'f1: {f_1}')

f1: 0.5220680958385876


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
predict_f = model.predict(features_valid)
f_1 = f1_score(target_valid, predict_f)
roc_auc = roc_auc_score(target_valid, predict_f)
print(f'f1: {f_1}')

f1: 0.5924713584288053


видно, что все модели страдают от дисбаланса так как наблюдается низкий f1, в особенности логистическая регрессия, так как она более чувствительна к дисбалансу






## Борьба с дисбалансом

теперь избавлюсь от дисбплпнса классов, продемонстрирую несколько способов это сделать

узнаю соотношения дисбаланса классов, чтобы знать во сколько увеличивать выборку

In [ ]:
class_ratio = len(target_train[target_train == 0]) / len(target_train[target_train == 1])
print(f"Соотношение классов: {class_ratio}:1")

Соотношение классов: 4.0133689839572195:1


для начала с помощью увеличения выборки

In [ ]:
from sklearn.utils import shuffle

def upsample(features, target, repeat):
  features_zeros = features[target == 0]
  features_ones = features[target == 1]
  target_zeros = target[target == 0]
  target_ones = target[target == 1]
  features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
  target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
  features_upsampled = shuffle(features_upsampled, random_state=12345)
  target_upsampled = shuffle(target_upsampled, random_state=12345)
  return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [ ]:
class_dist = target_upsampled.value_counts(normalize=True)
class_dist

,proportion
Exited,
0,0.500834
1,0.499166


In [ ]:
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_upsampled, target_upsampled)
predict_f = model.predict(features_valid)
f_1 = f1_score(target_valid, predict_f)
roc_auc = roc_auc_score(target_valid, predict_f)
print(f'f1: {f_1}')

f1: 0.5199240986717267


In [ ]:
model = DecisionTreeClassifier(random_state = 12345)
model.fit(features_upsampled, target_upsampled)
predict_f = model.predict(features_valid)
f_1 = f1_score(target_valid, predict_f)
roc_auc = roc_auc_score(target_valid, predict_f)
print(f'f1: {f_1}')

f1: 0.5099502487562189


In [ ]:
model = RandomForestClassifier(random_state = 12345)
model.fit(features_upsampled, target_upsampled)
predict_f = model.predict(features_valid)
f_1 = f1_score(target_valid, predict_f)
roc_auc = roc_auc_score(target_valid, predict_f)
print(f'f1: {f_1}')

f1: 0.5918367346938775


Можем сделать вывод, что увелечение выборки пошло на пользу логистической регрессии, а вот для модели дерева решений и случайного леса на этом наборе даннных не подходит и вероятно приводит к переобучению

теперь с помощью уменьшения выборки

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsample = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    return features_downsampled, target_downsample

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [ ]:
class_dist = target_downsampled.value_counts(normalize=True)
class_dist

,proportion
Exited,
0,0.50089
1,0.49911


In [ ]:
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_downsampled, target_downsampled)
predict_f = model.predict(features_valid)
f_1 = f1_score(target_valid, predict_f)
print(f'f1: {f_1}')

f1: 0.5065666041275797


In [ ]:
model = DecisionTreeClassifier(random_state = 12345)
model.fit(features_downsampled, target_downsampled)
predict_f = model.predict(features_valid)
f_1 = f1_score(target_valid, predict_f)
print(f'f1: {f_1}')

f1: 0.47965367965367967


In [ ]:
model = RandomForestClassifier(random_state = 12345)
model.fit(features_downsampled, target_downsampled)
predict_f = model.predict(features_valid)
f_1 = f1_score(target_valid, predict_f)
print(f'f1: {f_1}')

f1: 0.5885826771653543


Наблюдаем, что уменьшение выборки пошло на пользу логистической регрессии, но дало результат хуже, чем при увелечении выборки. Дерево решений и случайный лес уменьшили свои показатели, значит для них это не лучший спсоб

Теперь для лучших моделей подберем порог, при котором происходит наилусшее разделение классов!

In [ ]:
import numpy as np

In [ ]:
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_upsampled, target_upsampled)
predict_f = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc = roc_auc_score(target_valid, probabilities_one_valid)
print(f'roc_auc: {roc_auc:.4f}')
best_threshold = 0.5
best_f1 = 0
for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    current_f1 = f1_score(target_valid, predicted_valid)
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_threshold = threshold
print(f'порог: {best_threshold:.2f}, f1: {best_f1:.4f}')

roc_auc: 0.7943
порог: 0.56, f1: 0.5212


Видно, что классы сейчас более сбалансированны и на данный момент это лучшая оценка для логистической регрессии

In [ ]:
model = DecisionTreeClassifier(random_state = 12345)
model.fit(features_train, target_train)
predict_f = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc = roc_auc_score(target_valid, probabilities_one_valid)
print(f'roc_auc: {roc_auc:.4f}')
best_threshold = 0.5
best_f1 = 0
for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    current_f1 = f1_score(target_valid, predicted_valid)
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_threshold = threshold
print(f'порог: {best_threshold:.2f}, f1: {best_f1:.4f}')

roc_auc: 0.7035
порог: 0.00, f1: 0.5221


теперь можем подобрать глубину дерева, для улучшение метрик

In [ ]:
best_model = None
best_f1 = 0
count_deep = 0
best_threshold = 0.5
for i in range(1, 6):
  model = DecisionTreeClassifier(random_state = 12345, max_depth=i)
  model.fit(features_train, target_train)
  probabilities_valid = model.predict_proba(features_valid)
  probabilities_one_valid = probabilities_valid[:, 1]
  for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    current_f1 = f1_score(target_valid, predicted_valid)
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_threshold = threshold
        best_model = model
        best_depth = i
print(f'модель:{best_model}, глубина: {best_depth}, порог: {best_threshold:.2f}, f1: {best_f1:.4f}')

модель:DecisionTreeClassifier(max_depth=5, random_state=12345), глубина: 5, порог: 0.34, f1: 0.5924


На даннный момент это луччшая модель, которая найдена

In [ ]:
best_model = None
best_f1 = 0
count_tree = 0
best_threshold = 0.5
for est in range(1, 70):
  model = RandomForestClassifier(random_state=12345, n_estimators=est)
  model.fit(features_train, target_train)
  probabilities_valid = model.predict_proba(features_valid)
  probabilities_one_valid = probabilities_valid[:, 1]
  for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    current_f1 = f1_score(target_valid, predicted_valid)
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_threshold = threshold
        best_model = model
        count_tree = est
print(f'модель:{best_model}, кол-во деревьев: {best_depth}, порог: {best_threshold:.2f}, f1: {best_f1:.4f}')

модель:RandomForestClassifier(n_estimators=38, random_state=12345), кол-во деревьев: 5, порог: 0.36, f1: 0.6187


так же добавлю подбор глубины

In [ ]:
best_model = None
best_f1 = 0
best_estimators = 0
best_depth = 0
best_threshold = 0.5
for est in range(1, 70):
    for depth in range(1, 21):
        model = RandomForestClassifier(
            random_state=12345,
            n_estimators=est,
            max_depth=depth
        )
        model.fit(features_train, target_train)
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        for threshold in np.arange(0, 1, 0.02):
            predicted_valid = probabilities_one_valid > threshold
            current_f1 = f1_score(target_valid, predicted_valid)
            if current_f1 > best_f1:
                best_f1 = current_f1
                best_threshold = threshold
                best_model = model
                best_estimators = est
                best_depth = depth
print(f'модель: {best_model}, кол-во деревьев: {best_estimators}, глубина: {best_depth}, порог: {best_threshold:.2f}, f1: {best_f1:.4f}')

модель: RandomForestClassifier(max_depth=9, n_estimators=33, random_state=12345), кол-во деревьев: 9, глубина: 9, порог: 0.28, f1: 0.6526


## Тестирование модели

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=20)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
print(f'Accuracy модели случайного леса (с количеством деревьев:{count_tree}) на тестовой выборке составило:{result}')

In [ ]:
features_full_train, features_test, target_full_train, target_test

In [ ]:
final_model = RandomForestClassifier(
    random_state=12345,
    n_estimators=33,
    max_depth=9
)
final_model.fit(features_full_train, target_full_train)
probabilities_test = final_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
predicted_test = probabilities_one_test > 0.28
test_f1 = f1_score(target_test, predicted_test)
test_roc = roc_auc_score(target_test, probabilities_one_test)
print(f'roc на тесте:{test_roc:.4f}')
print(f'f1 на тесте: {test_f1:.4f}')

roc на тесте:0.8607
f1 на тесте: 0.6366


В результате внедрения разработанной модели машинного обучения для прогнозирования оттока клиентов, удалось значительно оптимизировать работу компании в направлении удержания ценных клиентов.<br>
<br>
Анализ метрик показал, что модель обладает высокой способностью к ранжированию клиентов по риску оттока<br> roc_auc = 0.8607, что позволяет с высокой точностью выявлять <br> наиболее вероятных к уходу.<br>
<br>
При установленном пороге вероятности в 0.28, модель демонстрирует сбалансированную эффективность <br>
f1_score = 0.6366, обеспечивая высокую полноту выявления потенциальных оттоков. Это означает, что можно своевременно предпринимать целевые действия для удержания значительной части клиентов, находящихся в зоне риска.<br>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [х]  Весь код выполняется без ошибок
- [х]  Ячейки с кодом расположены в порядке исполнения
- [х]  Выполнен шаг 1: данные подготовлены
- [х]  Выполнен шаг 2: задача исследована
    - [х]  Исследован баланс классов
    - [х]  Изучены модели без учёта дисбаланса
    - [х]  Написаны выводы по результатам исследования
- [х]  Выполнен шаг 3: учтён дисбаланс
    - [х]  Применено несколько способов борьбы с дисбалансом
    - [х]  Написаны выводы по результатам исследования
- [х]  Выполнен шаг 4: проведено тестирование
- [х]  Удалось достичь *F1*-меры не менее 0.59
- [х]  Исследована метрика *AUC-ROC*